In [1]:
%env DGLBACKEND=tensorflow

env: DGLBACKEND=tensorflow


In [2]:
import dgl.data
import tensorflow as tf
import tensorflow_addons as tfa
from dgl.nn import GraphConv
from sklearn.model_selection import train_test_split

2023-09-11 20:14:50.868803: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 20:14:50.907388: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-11 20:14:50.908017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-11 20:14:51.824358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-11 20:14:52.776435: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for ho

In [3]:
dataset = dgl.data.GINDataset("PROTEINS", self_loop=True)
print("node feature dimensionality:", dataset.dim_nfeats)
print("number of graph categories:", dataset.gclasses)
print("number of graphs in dataset:", len(dataset))

node feature dimensionality: 3
number of graph categories: 2
number of graphs in dataset: 1113


In [4]:
tv_dataset, test_dataset = train_test_split(dataset, shuffle=True, test_size=0.2)
train_dataset, val_dataset = train_test_split(tv_dataset, test_size=0.1)
print(len(train_dataset), len(val_dataset), len(test_dataset))

801 89 223


In [5]:
class GraphClassifier(tf.keras.Model):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GraphClassifier, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats, activation=tf.nn.relu)
        self.conv2 = GraphConv(h_feats, num_classes)

    def call(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = self.conv2(g, h)
        g.ndata["h"] = h
        return dgl.mean_nodes(g, "h")

In [6]:
def set_gpu_if_available():
    device="/cpu:0"
    gpus = tf.config.list_physical_devices("GPU")
    if len(gpus) > 0:
        device = gpus[0]
    return device

In [7]:
HIDDEN_SIZE = 16
BATCH_SIZE = 16
LEARNING_RATE = 1e-2
NUM_EPOCHS = 20

device = set_gpu_if_available()

def do_eval(model, dataset):
    total_acc, total_recs = 0, 0
    indexes = tf.data.Dataset.from_tensor_slices(range(len(dataset)))
    indexes = indexes.batch(batch_size=BATCH_SIZE)
    
    for batched_indexes in indexes:
        graphs, labels = zip(*[dataset[i] for i in batched_indexes])
        batched_graphs = dgl.batch(graphs)
        batched_labels = tf.convert_to_tensor(labels, dtype=tf.int64)
        batched_graphs = batched_graphs.to(device)
        logits = model(batched_graphs, batched_graphs.ndata["attr"])
        batched_preds = tf.math.argmax(logits, axis=1)
        acc = tf.reduce_sum(tf.cast(batched_preds == batched_labels, dtype=tf.float32))
        total_acc += acc.numpy().item()
        total_recs += len(batched_labels)
        return total_acc / total_recs

In [8]:
with tf.device(device):
    model = GraphClassifier(
        dataset.dim_nfeats, HIDDEN_SIZE, dataset.gclasses)
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    loss_fcn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    train_indexes = tf.data.Dataset.from_tensor_slices(range(len(train_dataset)))
    train_indexes = train_indexes.batch(batch_size=BATCH_SIZE)
    
    for epoch in range(NUM_EPOCHS):
        total_loss = 0
        for batched_indexes in train_indexes:
            with tf.GradientTape() as tape:
                graphs, labels = zip(*[train_dataset[i] for i in batched_indexes])
                batched_graphs = dgl.batch(graphs)
                batched_labels = tf.convert_to_tensor(labels, dtype=tf.int32)
                batched_graphs = batched_graphs.to(device)
                logits = model(batched_graphs, batched_graphs.ndata["attr"])
                loss = loss_fcn(batched_labels, logits)
            grads = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
            total_loss += loss.numpy().item()
    val_acc = do_eval(model, val_dataset)
    print("Epoch {:3d} | train_loss: {:.3f} | val_acc: {:.3f}".format(
    epoch, total_loss, val_acc))

Epoch  19 | train_loss: 31.343 | val_acc: 0.812


In [9]:
test_acc = do_eval(model, test_dataset)
print("test accuracy: {:.3f}".format(test_acc))

test accuracy: 0.750
